**Troubleshooting VEP Output and Filtering**

The VEP filtering procedure performed before came up with much less variants than expected for the entire human proteome. We need to find out some reason why this could be the case.

In [12]:
import os
import sys
import pandas as pd
from IPython.display import display

ROOT_DIR = os.environ.get('PYTHONPATH')

TROUBLESHOOT_PATH = os.path.join(ROOT_DIR, 'files/vep_troubleshoot')

VEP_OUTPUT_PATH = os.path.join(TROUBLESHOOT_PATH, 'uniprot_human_variants_vep_all.csv')

# keep dataframe in memory
vep_df = pd.read_csv(VEP_OUTPUT_PATH, sep='\t')

/var/folders/y3/mxnsyg_910x11q_t3z54wkbr0000gn/T/ipykernel_33917/3535052536.py:13: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  vep_df = pd.read_csv(VEP_OUTPUT_PATH, sep='\t')


In [3]:
display(vep_df)

,#Uploaded_variation,SWISSPROT,REF_ALLELE,Allele,Consequence,Protein_position,Amino_acids,CLIN_SIG,gnomADg_AF,CANONICAL
0,NC_000002.12:g.88978974G>A,A0A075B6H7.48,G,A,missense_variant,4,P/L,-,-,YES
1,NC_000002.12:g.88978975G>T,A0A075B6H7.48,G,T,missense_variant,4,P/T,-,-,YES
2,NC_000002.12:g.88978971G>A,A0A075B6H7.48,G,A,missense_variant,5,A/V,-,2.033e-05,YES
3,NC_000002.12:g.88978972C>T,A0A075B6H7.48,C,T,missense_variant,5,A/T,-,6.774e-06,YES
4,NC_000002.12:g.88978968T>G,A0A075B6H7.48,T,G,missense_variant,6,Q/P,-,6.773e-06,YES
...,...,...,...,...,...,...,...,...,...,...
10681953,NC_000001.11:g.158999996T>C,W6CW81.33,T,C,missense_variant,106,H/R,-,2.63e-05,YES
10681954,NC_000001.11:g.158999987G>C,W6CW81.33,G,C,missense_variant,109,S/C,-,-,YES
10681955,NC_000001.11:g.158999987G>T,W6CW81.33,G,T,missense_variant,109,S/Y,-,1.314e-05,YES
10681956,NC_000001.11:g.158999978G>A,W6CW81.33,G,A,missense_variant,112,A/V,-,-,YES


**Try to get rows with benign/pathogenic disregarding ambiguity**

In [96]:
benignRegex = r'(benign|-)'
justBenignRegex = r'(benign)'
pathogenicRegex = r'(pathogenic|damaging)'
consequenceRegex = r'(missense_variant)'

def evaluate_AF(AF):
   
   try:
      flt = float(AF.values)
      return flt
   except TypeError:
      return -1
   
def convert_AF(AF):
   
   try:
      return float(AF)
   except ValueError:
      return -1

# benign
vep_all_benign_df = vep_df[vep_df['CLIN_SIG'].str.contains(benignRegex, regex=True) & vep_df['Consequence'].str.contains(consequenceRegex, regex=True)]
benign_high_AF = vep_all_benign_df['gnomADg_AF'].apply(convert_AF) >= 0.01
vep_all_benign_df = vep_all_benign_df[vep_df['CLIN_SIG'].str.contains(justBenignRegex, regex=True) | benign_high_AF]

# pathogenic
vep_all_pathogenic_df = vep_df[vep_df['CLIN_SIG'].str.contains(pathogenicRegex, regex=True) & vep_df['Consequence'].str.contains(consequenceRegex, regex=True)]

VEP_BENIGN_OUTPUT = os.path.join(TROUBLESHOOT_PATH, 'vep_output_benign_variants.csv')
VEP_PATHOGENIC_OUTPUT = os.path.join(TROUBLESHOOT_PATH, 'vep_output_pathogenic_variants.csv')

vep_all_benign_df.to_csv(VEP_BENIGN_OUTPUT, sep='\t', index=False)
vep_all_pathogenic_df.to_csv(VEP_PATHOGENIC_OUTPUT, sep='\t', index=False)

display(vep_all_benign_df)
display(vep_all_pathogenic_df)

/var/folders/y3/mxnsyg_910x11q_t3z54wkbr0000gn/T/ipykernel_33917/1070662649.py:21: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  vep_all_benign_df = vep_df[vep_df['CLIN_SIG'].str.contains(benignRegex, regex=True) & vep_df['Consequence'].str.contains(consequenceRegex, regex=True)]
/var/folders/y3/mxnsyg_910x11q_t3z54wkbr0000gn/T/ipykernel_33917/1070662649.py:23: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  vep_all_benign_df = vep_all_benign_df[vep_df['CLIN_SIG'].str.contains(justBenignRegex, regex=True) | benign_high_AF]
/var/folders/y3/mxnsyg_910x11q_t3z54wkbr0000gn/T/ipykernel_33917/1070662649.py:23: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  vep_all_benign_df = vep_all_benign_df[vep_df['CLIN_SIG'].str.contains(justBenignRegex, regex=True) | benign_high_AF]
/var/folders/y3/m

,#Uploaded_variation,SWISSPROT,REF_ALLELE,Allele,Consequence,Protein_position,Amino_acids,CLIN_SIG,gnomADg_AF,CANONICAL
59,NC_000002.12:g.88978700A>G,A0A075B6H7.48,A,G,missense_variant,39,V/A,-,0.0208,YES
95,NC_000002.12:g.88978653T>A,A0A075B6H7.48,T,A,missense_variant,55,T/S,-,0.0113,YES
143,NC_000002.12:g.88978584T>C,A0A075B6H7.48,T,C,missense_variant,78,S/G,-,0.2179,YES
411,NC_000002.12:g.90190408C>T,A0A075B6H8.46,C,T,missense_variant,27,T/I,-,0.7535,YES
605,NC_000022.11:g.22201720G>C,A0A075B6I3.37,G,C,"missense_variant,splice_region_variant",16,G/R,-,0.01368,YES
...,...,...,...,...,...,...,...,...,...,...
10681651,NC_000008.11:g.119105892C>T,Q9Y6Z7.155,C,T,missense_variant,179,R/W,likely_benign,0.001565,YES
10681854,NC_000019.10:g.51704275G>A,W5XKT8.43,G,A,missense_variant,246,G/S,-,0.02386,YES
10681877,NC_000019.10:g.51704332T>A,W5XKT8.43,T,A,missense_variant,265,W/R,-,0.01797,YES
10681921,NC_000001.11:g.159000194A>T,W6CW81.33,A,T,missense_variant,40,L/Q,-,0.01915,YES


,#Uploaded_variation,SWISSPROT,REF_ALLELE,Allele,Consequence,Protein_position,Amino_acids,CLIN_SIG,gnomADg_AF,CANONICAL
9703,NC_000014.9:g.23104829G>A,A0A1B0GTW7.27,G,A,missense_variant,31,S/F,pathogenic,0.001065,YES
9922,NC_000014.9:g.23102250G>A,A0A1B0GTW7.27,G,A,missense_variant,384,S/L,pathogenic,1.971e-05,YES
14558,NC_000007.14:g.139161018A>G,A0AVF1.135,A,G,missense_variant,263,N/S,"likely_pathogenic,pathogenic",-,YES
14559,NC_000007.14:g.139161018A>G,A0AVF1.135,A,G,missense_variant,263,N/S,"likely_pathogenic,pathogenic",-,YES
22222,NC_000007.14:g.122303281G>A,A0PJY2.135,G,A,missense_variant,278,H/Y,pathogenic,-,YES
...,...,...,...,...,...,...,...,...,...,...
10680087,NC_000001.11:g.7677682C>T,Q9Y6Y1.171,C,T,missense_variant,955,R/W,likely_pathogenic,-,YES
10680187,NC_000001.11:g.7736507A>G,Q9Y6Y1.171,A,G,missense_variant,1077,Y/C,likely_pathogenic,-,YES
10680188,NC_000001.11:g.7736507A>G,Q9Y6Y1.171,A,G,missense_variant,1077,Y/C,likely_pathogenic,-,YES
10681645,NC_000008.11:g.119105885C>G,Q9Y6Z7.155,C,G,missense_variant,176,C/W,pathogenic,-,YES


**Check how many rows reduced when you exclude ambiguous mutations**

In [97]:
import re

ambiguousRegex = r'(uncertain|conflicting)'

vep_unamb_benign_df = vep_all_benign_df[~vep_all_benign_df['CLIN_SIG'].str.contains(ambiguousRegex, regex=True)]
vep_unamb_pathogenic_df = vep_all_pathogenic_df[~vep_all_pathogenic_df['CLIN_SIG'].str.contains(ambiguousRegex, regex=True)]

display(vep_unamb_benign_df)
display(vep_unamb_pathogenic_df)



/var/folders/y3/mxnsyg_910x11q_t3z54wkbr0000gn/T/ipykernel_33917/807473351.py:5: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  vep_unamb_benign_df = vep_all_benign_df[~vep_all_benign_df['CLIN_SIG'].str.contains(ambiguousRegex, regex=True)]
/var/folders/y3/mxnsyg_910x11q_t3z54wkbr0000gn/T/ipykernel_33917/807473351.py:6: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  vep_unamb_pathogenic_df = vep_all_pathogenic_df[~vep_all_pathogenic_df['CLIN_SIG'].str.contains(ambiguousRegex, regex=True)]


,#Uploaded_variation,SWISSPROT,REF_ALLELE,Allele,Consequence,Protein_position,Amino_acids,CLIN_SIG,gnomADg_AF,CANONICAL
59,NC_000002.12:g.88978700A>G,A0A075B6H7.48,A,G,missense_variant,39,V/A,-,0.0208,YES
95,NC_000002.12:g.88978653T>A,A0A075B6H7.48,T,A,missense_variant,55,T/S,-,0.0113,YES
143,NC_000002.12:g.88978584T>C,A0A075B6H7.48,T,C,missense_variant,78,S/G,-,0.2179,YES
411,NC_000002.12:g.90190408C>T,A0A075B6H8.46,C,T,missense_variant,27,T/I,-,0.7535,YES
605,NC_000022.11:g.22201720G>C,A0A075B6I3.37,G,C,"missense_variant,splice_region_variant",16,G/R,-,0.01368,YES
...,...,...,...,...,...,...,...,...,...,...
10681651,NC_000008.11:g.119105892C>T,Q9Y6Z7.155,C,T,missense_variant,179,R/W,likely_benign,0.001565,YES
10681854,NC_000019.10:g.51704275G>A,W5XKT8.43,G,A,missense_variant,246,G/S,-,0.02386,YES
10681877,NC_000019.10:g.51704332T>A,W5XKT8.43,T,A,missense_variant,265,W/R,-,0.01797,YES
10681921,NC_000001.11:g.159000194A>T,W6CW81.33,A,T,missense_variant,40,L/Q,-,0.01915,YES


,#Uploaded_variation,SWISSPROT,REF_ALLELE,Allele,Consequence,Protein_position,Amino_acids,CLIN_SIG,gnomADg_AF,CANONICAL
9703,NC_000014.9:g.23104829G>A,A0A1B0GTW7.27,G,A,missense_variant,31,S/F,pathogenic,0.001065,YES
9922,NC_000014.9:g.23102250G>A,A0A1B0GTW7.27,G,A,missense_variant,384,S/L,pathogenic,1.971e-05,YES
14558,NC_000007.14:g.139161018A>G,A0AVF1.135,A,G,missense_variant,263,N/S,"likely_pathogenic,pathogenic",-,YES
14559,NC_000007.14:g.139161018A>G,A0AVF1.135,A,G,missense_variant,263,N/S,"likely_pathogenic,pathogenic",-,YES
22222,NC_000007.14:g.122303281G>A,A0PJY2.135,G,A,missense_variant,278,H/Y,pathogenic,-,YES
...,...,...,...,...,...,...,...,...,...,...
10680087,NC_000001.11:g.7677682C>T,Q9Y6Y1.171,C,T,missense_variant,955,R/W,likely_pathogenic,-,YES
10680187,NC_000001.11:g.7736507A>G,Q9Y6Y1.171,A,G,missense_variant,1077,Y/C,likely_pathogenic,-,YES
10680188,NC_000001.11:g.7736507A>G,Q9Y6Y1.171,A,G,missense_variant,1077,Y/C,likely_pathogenic,-,YES
10681645,NC_000008.11:g.119105885C>G,Q9Y6Z7.155,C,G,missense_variant,176,C/W,pathogenic,-,YES


**Exclude Non-reviewed Proteins**

In [98]:
REVIEWED_UNIPROT_PATH = os.path.join(ROOT_DIR, 'files/uniprot_human_reviewed_lt_2700aa.csv')
from utils import readFile_as_generator

REVIEWED_UNIPROT = {}

def std_uniprot(uniprot: str):

   pattern = r'\..*'
   
   return uniprot.replace(pattern, '', regex=True)

def check_protein(uniprot):
   try:
      value = REVIEWED_UNIPROT[str(uniprot)]
      return value
   except KeyError:
      return False
   
for protein in readFile_as_generator(REVIEWED_UNIPROT_PATH):
   REVIEWED_UNIPROT[protein.strip('\n')] = True

print(REVIEWED_UNIPROT)

# standardize SWISSPROT column
vep_unamb_benign_df['SWISSPROT'] = std_uniprot(vep_unamb_benign_df['SWISSPROT'])
vep_unamb_pathogenic_df['SWISSPROT'] = std_uniprot(vep_unamb_pathogenic_df['SWISSPROT'])

reviewed_benign_rows = vep_unamb_benign_df['SWISSPROT'].apply(check_protein)
reviewed_pathogenic_rows = vep_unamb_pathogenic_df['SWISSPROT'].apply(check_protein)

vep_benign_reviewed_df = vep_unamb_benign_df[reviewed_benign_rows]
vep_pathogenic_reviewed_df = vep_unamb_pathogenic_df[reviewed_pathogenic_rows]

display(vep_benign_reviewed_df)
display(vep_pathogenic_reviewed_df)

{'A0A0B4J2F0': True, 'A0A0C5B5G6': True, 'A0A0K2S4Q6': True, 'A0A0U1RRE5': True, 'A0A1B0GTW7': True, 'A0A5B9': True, 'A0AV02': True, 'A0AV96': True, 'A0AVF1': True, 'A0AVI4': True, 'A0AVK6': True, 'A0AVT1': True, 'A0FGR8': True, 'A0FGR9': True, 'A0JLT2': True, 'A0JNW5': True, 'A0JP26': True, 'A0M8Q6': True, 'A0MZ66': True, 'A0PJK1': True, 'A0PJY2': True, 'A0PK00': True, 'A0PK11': True, 'A1A4S6': True, 'A1A4Y4': True, 'A1A5B4': True, 'A1A5C7': True, 'A1E959': True, 'A1KXE4': True, 'A1KZ92': True, 'A1L0T0': True, 'A1L167': True, 'A1L190': True, 'A1L390': True, 'A1L3X0': True, 'A1L4H1': True, 'A1L4K1': True, 'A1X283': True, 'A1XBS5': True, 'A1Z1Q3': True, 'A2A288': True, 'A2A2Y4': True, 'A2IDD5': True, 'A2PYH4': True, 'A2RRP1': True, 'A2RTX5': True, 'A2RU14': True, 'A2RU30': True, 'A2RU49': True, 'A2RUB6': True, 'A2RUC4': True, 'A2RUH7': True, 'A2VDJ0': True, 'A3KMH1': True, 'A4D126': True, 'A4D161': True, 'A4D1B5': True, 'A4D1E9': True, 'A4D1P6': True, 'A4D1T9': True, 'A4D2B0': True, 'A4

/var/folders/y3/mxnsyg_910x11q_t3z54wkbr0000gn/T/ipykernel_33917/4287797269.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vep_unamb_benign_df['SWISSPROT'] = std_uniprot(vep_unamb_benign_df['SWISSPROT'])
/var/folders/y3/mxnsyg_910x11q_t3z54wkbr0000gn/T/ipykernel_33917/4287797269.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vep_unamb_pathogenic_df['SWISSPROT'] = std_uniprot(vep_unamb_pathogenic_df['SWISSPROT'])


,#Uploaded_variation,SWISSPROT,REF_ALLELE,Allele,Consequence,Protein_position,Amino_acids,CLIN_SIG,gnomADg_AF,CANONICAL
59,NC_000002.12:g.88978700A>G,A0A075B6H7,A,G,missense_variant,39,V/A,-,0.0208,YES
95,NC_000002.12:g.88978653T>A,A0A075B6H7,T,A,missense_variant,55,T/S,-,0.0113,YES
143,NC_000002.12:g.88978584T>C,A0A075B6H7,T,C,missense_variant,78,S/G,-,0.2179,YES
411,NC_000002.12:g.90190408C>T,A0A075B6H8,C,T,missense_variant,27,T/I,-,0.7535,YES
605,NC_000022.11:g.22201720G>C,A0A075B6I3,G,C,"missense_variant,splice_region_variant",16,G/R,-,0.01368,YES
...,...,...,...,...,...,...,...,...,...,...
10681651,NC_000008.11:g.119105892C>T,Q9Y6Z7,C,T,missense_variant,179,R/W,likely_benign,0.001565,YES
10681854,NC_000019.10:g.51704275G>A,W5XKT8,G,A,missense_variant,246,G/S,-,0.02386,YES
10681877,NC_000019.10:g.51704332T>A,W5XKT8,T,A,missense_variant,265,W/R,-,0.01797,YES
10681921,NC_000001.11:g.159000194A>T,W6CW81,A,T,missense_variant,40,L/Q,-,0.01915,YES


,#Uploaded_variation,SWISSPROT,REF_ALLELE,Allele,Consequence,Protein_position,Amino_acids,CLIN_SIG,gnomADg_AF,CANONICAL
9703,NC_000014.9:g.23104829G>A,A0A1B0GTW7,G,A,missense_variant,31,S/F,pathogenic,0.001065,YES
9922,NC_000014.9:g.23102250G>A,A0A1B0GTW7,G,A,missense_variant,384,S/L,pathogenic,1.971e-05,YES
14558,NC_000007.14:g.139161018A>G,A0AVF1,A,G,missense_variant,263,N/S,"likely_pathogenic,pathogenic",-,YES
14559,NC_000007.14:g.139161018A>G,A0AVF1,A,G,missense_variant,263,N/S,"likely_pathogenic,pathogenic",-,YES
22222,NC_000007.14:g.122303281G>A,A0PJY2,G,A,missense_variant,278,H/Y,pathogenic,-,YES
...,...,...,...,...,...,...,...,...,...,...
10680087,NC_000001.11:g.7677682C>T,Q9Y6Y1,C,T,missense_variant,955,R/W,likely_pathogenic,-,YES
10680187,NC_000001.11:g.7736507A>G,Q9Y6Y1,A,G,missense_variant,1077,Y/C,likely_pathogenic,-,YES
10680188,NC_000001.11:g.7736507A>G,Q9Y6Y1,A,G,missense_variant,1077,Y/C,likely_pathogenic,-,YES
10681645,NC_000008.11:g.119105885C>G,Q9Y6Z7,C,G,missense_variant,176,C/W,pathogenic,-,YES


**Now we need to drop all the unnecessary columns and format `Amino_acids` column into `WT` and `Mut` columns**

In [99]:
cols_to_drop = ['#Uploaded_variation', 'REF_ALLELE', 'Allele', 'Consequence', 'Amino_acids', 'CLIN_SIG', 'gnomADg_AF', 'CANONICAL']

def get_WT(mutation):
   
   mutationStr = str(mutation)
   
   aa = mutationStr.split('/')
   
   return aa[0]

def get_Mut(mutation):
   
   mutationStr = str(mutation)
   
   try:
      aa = mutationStr.split('/')[1]
      return aa
   except IndexError:
      return mutation

# benign
vep_benign_reviewed_df['WT'] = vep_benign_reviewed_df['Amino_acids'].apply(get_WT)
vep_benign_reviewed_df['Mut'] = vep_benign_reviewed_df['Amino_acids'].apply(get_Mut)

# pathogenic
vep_pathogenic_reviewed_df['WT'] = vep_pathogenic_reviewed_df['Amino_acids'].apply(get_WT)
vep_pathogenic_reviewed_df['Mut'] = vep_pathogenic_reviewed_df['Amino_acids'].apply(get_Mut)

vep_benign_reviewed_df = vep_benign_reviewed_df.drop(columns=cols_to_drop).rename(columns={'SWISSPROT': 'uniprot', 'Protein_position': 'position'})
vep_pathogenic_reviewed_df = vep_pathogenic_reviewed_df.drop(columns=cols_to_drop).rename(columns={'SWISSPROT': 'uniprot', 'Protein_position': 'position'})

display(vep_benign_reviewed_df)
display(vep_pathogenic_reviewed_df)

OUTPUT_PATH = os.path.join(ROOT_DIR, 'synced_files/vep_variants')

BENIGN_OUTFILE = os.path.join(OUTPUT_PATH, 'vep_benign_variants.csv')
PATHOG_OUTFILE = os.path.join(OUTPUT_PATH, 'vep_pathogenic_variants.csv')

vep_benign_reviewed_df.to_csv(BENIGN_OUTFILE, sep='\t', index=False)
vep_pathogenic_reviewed_df.to_csv(PATHOG_OUTFILE, sep='\t', index=False)

/var/folders/y3/mxnsyg_910x11q_t3z54wkbr0000gn/T/ipykernel_33917/4002592143.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vep_benign_reviewed_df['WT'] = vep_benign_reviewed_df['Amino_acids'].apply(get_WT)
/var/folders/y3/mxnsyg_910x11q_t3z54wkbr0000gn/T/ipykernel_33917/4002592143.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vep_benign_reviewed_df['Mut'] = vep_benign_reviewed_df['Amino_acids'].apply(get_Mut)
/var/folders/y3/mxnsyg_910x11q_t3z54wkbr0000gn/T/ipykernel_33917/4002592143.py:26: Set

,uniprot,position,WT,Mut
59,A0A075B6H7,39,V,A
95,A0A075B6H7,55,T,S
143,A0A075B6H7,78,S,G
411,A0A075B6H8,27,T,I
605,A0A075B6I3,16,G,R
...,...,...,...,...
10681651,Q9Y6Z7,179,R,W
10681854,W5XKT8,246,G,S
10681877,W5XKT8,265,W,R
10681921,W6CW81,40,L,Q


,uniprot,position,WT,Mut
9703,A0A1B0GTW7,31,S,F
9922,A0A1B0GTW7,384,S,L
14558,A0AVF1,263,N,S
14559,A0AVF1,263,N,S
22222,A0PJY2,278,H,Y
...,...,...,...,...
10680087,Q9Y6Y1,955,R,W
10680187,Q9Y6Y1,1077,Y,C
10680188,Q9Y6Y1,1077,Y,C
10681645,Q9Y6Z7,176,C,W
